In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed


Using TensorFlow backend.


In [2]:
DATA_DIR = '../../../analysis/data/nbmodel_templates.csv'
BATCH_SIZE = 50
HIDDEN_DIM = 200
SEQ_LENGTH = 5
WEIGHTS = ''

GENERATE_LENGTH = 10
LAYER_NUM = 2

In [3]:
reports = []

with open(DATA_DIR, encoding='utf-8-sig') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            reports.append(row[2].replace('"', '').replace("[", '').replace(']', '').split(","))
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are report, report_chunks, template_chunks, templates
Processed 1953 lines.


In [4]:
data = np.concatenate(reports)
len(data)

6756

In [5]:
# method for preparing the training data
def load_data(data, seq_length):
    chunks = list(set(data))
    VOCAB_SIZE = len(chunks)

    print('Data length: {} chunks'.format(len(data)))
    print('Vocabulary size: {} chunks'.format(VOCAB_SIZE))

    ix_to_chunk = {ix:chunk for ix, chunk in enumerate(chunks)}
    chunk_to_ix = {chunk:ix for ix, chunk in enumerate(chunks)}
    
    num_seq = int(np.floor((len(data) - 1) / seq_length))

    X = np.zeros((num_seq, seq_length, VOCAB_SIZE))
    y = np.zeros((num_seq, seq_length, VOCAB_SIZE))
    for i in range(0, num_seq):
        X_sequence = data[i*seq_length:(i+1)*seq_length]
        X_sequence_ix = [chunk_to_ix[value] for value in X_sequence]
        input_sequence = np.zeros((seq_length, VOCAB_SIZE))
        for j in range(seq_length):
            input_sequence[j][X_sequence_ix[j]] = 1.
            X[i] = input_sequence

        y_sequence = data[i*seq_length+1:(i+1)*seq_length+1]
        y_sequence_ix = [chunk_to_ix[value] for value in y_sequence]
        target_sequence = np.zeros((seq_length, VOCAB_SIZE))
        for j in range(seq_length):
            target_sequence[j][y_sequence_ix[j]] = 1.
            y[i] = target_sequence
    return X, y, VOCAB_SIZE, ix_to_chunk

In [6]:
# Creating training data
X, y, VOCAB_SIZE, ix_to_chunk = load_data(data, SEQ_LENGTH)

Data length: 6756 chunks
Vocabulary size: 959 chunks


In [7]:
# method for generating text
def generate_text(model, length, vocab_size, ix_to_chunk):
    # starting with random chunk
    ix = [np.random.randint(vocab_size)]
    y_chunk = [ix_to_chunk[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted chunk to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_chunk[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_chunk.append(ix_to_chunk[ix[-1]])
    return ('').join(y_chunk)

In [8]:
# x = np.zeros((1, GENERATE_LENGTH, VOCAB_SIZE))

In [9]:
# model.predict(x[:, :5, :])

In [10]:
# Creating and compiling the Network
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
  model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [11]:
# Generate some sample before training to know how bad it is!
generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_chunk)

 in the Ravens' ${opp_score}-${team_score}'${player_name} caught 3 of 5 targets for 34 yards ' 'adding ${rush_attempts} carries for ${rush_yards} additional yards.' 'managed ${receptions} catch for ${rec_yards} yards on ${rec_targets} targets ' 'adding ${receptions} catches for ${rec_yards} yards ''Undrafted rookie ${player_name} ' 'vacuumed ${receptions} of ${rec_targets} targets for a team-high ${rec_yards} yards ' 'vacuumed ${receptions} of ${rec_targets} targets for a team-high ${rec_yards} yards ' 'vacuumed ${receptions} of ${rec_targets} targets for a team-high ${rec_yards} yards ' in ${team}'s week ${week} destruction from the ${opp}.

" in the Ravens' ${opp_score}-${team_score}'${player_name} caught 3 of 5 targets for 34 yards ' 'adding ${rush_attempts} carries for ${rush_yards} additional yards.' 'managed ${receptions} catch for ${rec_yards} yards on ${rec_targets} targets ' 'adding ${receptions} catches for ${rec_yards} yards ''Undrafted rookie ${player_name} ' 'vacuumed ${receptions} of ${rec_targets} targets for a team-high ${rec_yards} yards ' 'vacuumed ${receptions} of ${rec_targets} targets for a team-high ${rec_yards} yards ' 'vacuumed ${receptions} of ${rec_targets} targets for a team-high ${rec_yards} yards ' in ${team}'s week ${week} destruction from the ${opp}. and ${pass_int} interceptions '"

In [12]:
# Training if there is no trained weights specified
if not WEIGHTS == '':
  model.load_weights(WEIGHTS)
  nb_epoch = int(WEIGHTS[WEIGHTS.rfind('_') + 1:WEIGHTS.find('.')])
else:
  nb_epoch = 0

if WEIGHTS == '':
  while True:
    print('\n\nEpoch: {}\n'.format(nb_epoch))
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, nb_epoch=1)
    nb_epoch += 1
    #generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
    if nb_epoch % 10 == 0:
      model.save_weights('checkpoint_layer_{}_hidden_{}_epoch_{}.hdf5'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch))
# Else, loading the trained weights and perform generation only
elif WEIGHTS == '':
  # Loading the trained weights
  model.load_weights(WEIGHTS)
  generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
  print('\n\n')
else:
  print('\n\nNothing to do!')



Epoch: 0



c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\site-packages\ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/1
1351/1351 [==============================] - 2s 2ms/step - loss: 5.6347


Epoch: 1

Epoch 1/1
1351/1351 [==============================] - 1s 638us/step - loss: 4.7436 0s - loss: 4.742


Epoch: 2

Epoch 1/1
1351/1351 [==============================] - 1s 631us/step - loss: 4.5320


Epoch: 3

Epoch 1/1
1351/1351 [==============================] - 1s 657us/step - loss: 4.3424


Epoch: 4

Epoch 1/1
1351/1351 [==============================] - 1s 805us/step - loss: 4.2571


Epoch: 5

Epoch 1/1
1351/1351 [==============================] - 1s 728us/step - loss: 4.1934


Epoch: 6

Epoch 1/1
1351/1351 [==============================] - 1s 729us/step - loss: 4.0733


Epoch: 7

Epoch 1/1
1351/1351 [==============================] - 1s 651us/step - loss: 3.9622


Epoch: 8

Epoch 1/1
1351/1351 [==============================] - 1s 664us/step - loss: 3.8244


Epoch: 9

Epoch 1/1
1351/1351 [==============================] - 1s 672us/step - loss: 3.6884


Epoch: 10

Epoch 1/1
1351/1351 [===

1351/1351 [==============================] - 1s 560us/step - loss: 1.1775


Epoch: 85

Epoch 1/1
1351/1351 [==============================] - 1s 582us/step - loss: 1.1650


Epoch: 86

Epoch 1/1
1351/1351 [==============================] - 1s 673us/step - loss: 1.1562


Epoch: 87

Epoch 1/1
1351/1351 [==============================] - 1s 785us/step - loss: 1.1422


Epoch: 88

Epoch 1/1
1351/1351 [==============================] - 1s 622us/step - loss: 1.1278


Epoch: 89

Epoch 1/1
1351/1351 [==============================] - 1s 602us/step - loss: 1.1174


Epoch: 90

Epoch 1/1
1351/1351 [==============================] - 1s 574us/step - loss: 1.1109


Epoch: 91

Epoch 1/1
1351/1351 [==============================] - 1s 574us/step - loss: 1.0928


Epoch: 92

Epoch 1/1
1351/1351 [==============================] - 1s 565us/step - loss: 1.0953


Epoch: 93

Epoch 1/1
1351/1351 [==============================] - 1s 562us/step - loss: 1.0763


Epoch: 94

Epoch 1/1
1351/1351 [===================



Epoch: 168

Epoch 1/1
1351/1351 [==============================] - 1s 573us/step - loss: 0.7787


Epoch: 169

Epoch 1/1
1351/1351 [==============================] - 1s 578us/step - loss: 0.7757


Epoch: 170

Epoch 1/1
1351/1351 [==============================] - 1s 574us/step - loss: 0.7731


Epoch: 171

Epoch 1/1
1351/1351 [==============================] - 1s 592us/step - loss: 0.7744


Epoch: 172

Epoch 1/1
1351/1351 [==============================] - 1s 566us/step - loss: 0.7721


Epoch: 173

Epoch 1/1
1351/1351 [==============================] - 1s 570us/step - loss: 0.7711


Epoch: 174

Epoch 1/1
1351/1351 [==============================] - 1s 565us/step - loss: 0.7749


Epoch: 175

Epoch 1/1
1351/1351 [==============================] - 1s 573us/step - loss: 0.7691


Epoch: 176

Epoch 1/1
1351/1351 [==============================] - 1s 571us/step - loss: 0.7682


Epoch: 177

Epoch 1/1
1351/1351 [==============================] - 1s 574us/step - loss: 0.7667


Epoch: 178

Epoch 

...

1351/1351 [==============================] - 1s 568us/step - loss: 0.7348


Epoch: 335

Epoch 1/1
1351/1351 [==============================] - 1s 559us/step - loss: 0.7338


Epoch: 336

Epoch 1/1
1351/1351 [==============================] - 1s 567us/step - loss: 0.7339


Epoch: 337

Epoch 1/1
1351/1351 [==============================] - 1s 570us/step - loss: 0.7335


Epoch: 338

Epoch 1/1
1351/1351 [==============================] - 1s 563us/step - loss: 0.7361


Epoch: 339

Epoch 1/1
1351/1351 [==============================] - 1s 592us/step - loss: 0.7336


Epoch: 340

Epoch 1/1
1351/1351 [==============================] - 1s 591us/step - loss: 0.7331


Epoch: 341

Epoch 1/1
1351/1351 [==============================] - 1s 592us/step - loss: 0.7348


Epoch: 342

Epoch 1/1
1351/1351 [==============================] - 1s 574us/step - loss: 0.7344


Epoch: 343

Epoch 1/1
1351/1351 [==============================] - 1s 575us/step - loss: 0.7330


Epoch: 344

Epoch 1/1
1351/1351 [=========

1351/1351 [==============================] - 1s 564us/step - loss: 0.7270


Epoch: 418

Epoch 1/1
1351/1351 [==============================] - 1s 567us/step - loss: 0.7296


Epoch: 419

Epoch 1/1
1351/1351 [==============================] - 1s 577us/step - loss: 0.7289


Epoch: 420

Epoch 1/1
1351/1351 [==============================] - 1s 594us/step - loss: 0.7274


Epoch: 421

Epoch 1/1
1351/1351 [==============================] - 1s 611us/step - loss: 0.7275


Epoch: 422

Epoch 1/1
1351/1351 [==============================] - 1s 575us/step - loss: 0.7268


Epoch: 423

Epoch 1/1
1351/1351 [==============================] - 1s 579us/step - loss: 0.7271


Epoch: 424

Epoch 1/1
1351/1351 [==============================] - 1s 573us/step - loss: 0.7287


Epoch: 425

Epoch 1/1
1351/1351 [==============================] - 1s 578us/step - loss: 0.7298


Epoch: 426

Epoch 1/1
1351/1351 [==============================] - 1s 574us/step - loss: 0.7288


Epoch: 427

Epoch 1/1
1351/1351 [=========

1351/1351 [==============================] - 1s 575us/step - loss: 0.7239


Epoch: 501

Epoch 1/1
1351/1351 [==============================] - 1s 585us/step - loss: 0.7221


Epoch: 502

Epoch 1/1
1351/1351 [==============================] - 1s 568us/step - loss: 0.7213


Epoch: 503

Epoch 1/1
1351/1351 [==============================] - 1s 573us/step - loss: 0.7200


Epoch: 504

Epoch 1/1
1351/1351 [==============================] - 1s 582us/step - loss: 0.7218


Epoch: 505

Epoch 1/1
1351/1351 [==============================] - 1s 580us/step - loss: 0.7228


Epoch: 506

Epoch 1/1
1351/1351 [==============================] - 1s 571us/step - loss: 0.7216


Epoch: 507

Epoch 1/1
1351/1351 [==============================] - 1s 564us/step - loss: 0.7223


Epoch: 508

Epoch 1/1
1351/1351 [==============================] - 1s 582us/step - loss: 0.7229


Epoch: 509

Epoch 1/1
1351/1351 [==============================] - 1s 568us/step - loss: 0.7227


Epoch: 510

Epoch 1/1
1351/1351 [=========

1351/1351 [==============================] - 1s 605us/step - loss: 0.7192


Epoch: 584

Epoch 1/1
1351/1351 [==============================] - 1s 594us/step - loss: 0.7172


Epoch: 585

Epoch 1/1
1351/1351 [==============================] - 1s 604us/step - loss: 0.7182


Epoch: 586

Epoch 1/1
1351/1351 [==============================] - 1s 594us/step - loss: 0.7172


Epoch: 587

Epoch 1/1
 150/1351 [==>...........................] - ETA: 0s - loss: 0.7098

KeyboardInterrupt: 

In [14]:
generate_text(model, 100, VOCAB_SIZE, ix_to_chunk)

 on his lone target ${game_dow} in the ${team}' week ${week} victory over ${opp}.'${player_name} ' 'caught ${receptions} of ${rec_targets} targets for ${rec_yards} yards and a touchdown ' in the ${team}' week ${week} win over the ${opp}.'${player_name} ' 'hauled in just ${receptions} of ${rec_targets} targets for ${rec_yards} yards ' 'in week ${week} against the ${opp}.''${player_name} ''${player_name} ' 'hauled in ${receptions} of ${rec_targets} targets for ${rec_yards} yards and ${rec_td} touchdowns ' in the ${team}' week ${week} win over the ${opp}.'${player_name} ' in the ${team}' week ${week} win over the ${opp}. in the ${team}' week ${week} win over the ${opp}. in the ${team}' week ${week} win over the ${opp}. adding a rushing score. also losing 2 fumbles. also losing 2 fumbles. also losing 2 fumbles. also losing 2 fumbles. 'for ${rush_yards} yards ' 'for ${rush_yards} yards ''${player_name} ' 'for ${rush_yards} yards ' 'for ${rush_yards} yards ''${player_name} ''${player_name} '

" on his lone target ${game_dow} in the ${team}' week ${week} victory over ${opp}.'${player_name} ' 'caught ${receptions} of ${rec_targets} targets for ${rec_yards} yards and a touchdown ' in the ${team}' week ${week} win over the ${opp}.'${player_name} ' 'hauled in just ${receptions} of ${rec_targets} targets for ${rec_yards} yards ' 'in week ${week} against the ${opp}.''${player_name} ''${player_name} ' 'hauled in ${receptions} of ${rec_targets} targets for ${rec_yards} yards and ${rec_td} touchdowns ' in the ${team}' week ${week} win over the ${opp}.'${player_name} ' in the ${team}' week ${week} win over the ${opp}. in the ${team}' week ${week} win over the ${opp}. in the ${team}' week ${week} win over the ${opp}. adding a rushing score. also losing 2 fumbles. also losing 2 fumbles. also losing 2 fumbles. also losing 2 fumbles. 'for ${rush_yards} yards ' 'for ${rush_yards} yards ''${player_name} ' 'for ${rush_yards} yards ' 'for ${rush_yards} yards ''${player_name} ''${player_name} 